In [1]:
! pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 776.5 kB/s eta 0:00:0000:0100:01
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)


In [6]:
! pip install pyproj


  Using cached pyproj-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)


In [4]:
import rasterio

def get_value_at_lon_lat(file_path, lon, lat):
    with rasterio.open(file_path) as src:
        # Convert geographic coordinates to raster coordinates
        row, col = src.index(lon, lat)
        
        # Read the raster values at the given coordinates
        value = src.read(1)[row, col]  # Assuming the value is in the first band
        return value

# Example usage
file_path = 'Geospatial Data/dar_nitrogen.tif'

longitude = 39.2397
latitude = -6.8190 
value = get_value_at_lon_lat(file_path, longitude, latitude)
print(f'The raster value at longitude {longitude} and latitude {latitude} is {value}')


IndexError: index -24531 is out of bounds for axis 0 with size 1942

In [15]:
import rasterio

def check_crs(file_path):
    with rasterio.open(file_path) as src:
        print("CRS of the raster:", src.crs)


check_crs(file_path)


CRS of the raster: EPSG:3857


In [10]:
from pyproj import Transformer

def transform_coords(lon, lat, target_crs):
    transformer = Transformer.from_crs("epsg:4326", target_crs, always_xy=True)
    return transformer.transform(lon, lat)

def get_value_at_lon_lat(file_path, lon, lat):
    with rasterio.open(file_path) as src:
        # Transform coordinates if necessary
        if src.crs.to_string() != 'EPSG:4326':
            lon, lat = transform_coords(lon, lat, src.crs.to_string())
        
        # Convert geographic coordinates to raster coordinates
        row, col = src.index(lon, lat)
        
        # Ensure the indices are within the raster dimensions
        if not (0 <= row < src.height and 0 <= col < src.width):
            return "Coordinates are outside the raster bounds"
        
        # Read the raster values at the given coordinates
        value = src.read(1)[row, col]  # Assuming the value is in the first band
        return value

# Example usage
longitude = 39.2397
latitude = -6.8190 
file_path = 'Geospatial Data/dar_ph.tif'
value = get_value_at_lon_lat(file_path, longitude, latitude)

print(f'The raster value at longitude {longitude} and latitude {latitude} is {value}')


The raster value at longitude 39.2397 and latitude -6.819 is 67


In [14]:
import rasterio
from pyproj import Transformer

def transform_coords(lon, lat, src_crs):
    # Transformer to convert from geographic (WGS 84) to the raster's CRS
    transformer = Transformer.from_crs("EPSG:4326", src_crs, always_xy=True)
    return transformer.transform(lon, lat)

def get_value_and_metadata_at_lon_lat(file_path, lon, lat):
    with rasterio.open(file_path) as src:
        # Transform input longitude and latitude to the coordinate system of the raster
        lon, lat = transform_coords(lon, lat, src.crs)

        # Convert transformed geographic coordinates to raster coordinates
        row, col = src.index(lon, lat)

        # Check if the indices are within the bounds of the raster
        if not (0 <= row < src.height and 0 <= col < src.width):
            return "Coordinates are outside the raster bounds", None
        
        # Read the raster values at the given coordinates
        value = src.read(1)[row, col]  # Assuming the value is in the first band

        # Read extra metadata (tags) from the raster
        # You can also access src.tags(band_index) if metadata is band-specific
        print(src)
        metadata = src.tags()

        return value, metadata

# Example usage
file_path = 'Geospatial Data/dar_nitrogen.tif'
longitude = 39.2397
latitude = -6.8190 
value, metadata = get_value_and_metadata_at_lon_lat(file_path, longitude, latitude)
print(f"The raster value at longitude {longitude} and latitude {latitude} is {value}")
print("Metadata:", metadata)


<open DatasetReader name='Geospatial Data/dar_nitrogen.tif' mode='r'>
The raster value at longitude 39.2397 and latitude -6.819 is 74
Metadata: {'AREA_OR_POINT': 'Area'}


In [16]:
dataset = rasterio.open('Geospatial Data/dar_nitrogen.tif')

In [17]:
dataset.name

'Geospatial Data/dar_nitrogen.tif'

In [18]:
dataset.count

4

In [19]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'uint8', 2: 'uint8', 3: 'uint8', 4: 'uint8'}

In [40]:
dataset

{'AREA_OR_POINT': 'Area'}

In [37]:
band1

[4339327.192356968, 30.0, 0.0, -735916.8695475701, 0.0, -30.0]

In [30]:
dataset.extra_fields["back-transformation"]

AttributeError: 'DatasetReader' object has no attribute 'extra_fields'

In [ ]:
https://api.mapbox.com/search/geocode/v6/reverse?longitude={longitude}&latitude={latitude}

In [2]:
import requests

def reverse_geocode(latitude, longitude, access_token):
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{32},{-6}.json?access_token=pk.eyJ1IjoiZ29kc29ubnR1bmdpIiwiYSI6ImNsdjhjazB6cDAxcjYyaXFsaTMzdjV0eWoifQ.iPSTQY-h8G7pB3u7GVktPA&permanent=true"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print(data)
        # The 'features' list contains address information, with the most relevant usually first
        if data['features']:
            return data['features'][0]['place_name']
    return None

# Usage
latitude = 40.7128
longitude = -74.0060
access_token = "your_mapbox_access_token_here"
address = reverse_geocode(latitude, longitude, access_token)
print(address)

{'type': 'FeatureCollection', 'query': [32, -6], 'features': [{'id': 'region.99560', 'type': 'Feature', 'place_type': ['region'], 'relevance': 1, 'properties': {'mapbox_id': 'dXJuOm1ieHBsYzpBWVRv', 'wikidata': 'Q2682731', 'short_code': 'TZ-28'}, 'text': 'Katavi', 'place_name': 'Katavi, Tanzania', 'bbox': [29.903673, -7.679326, 32.759552, -5.135649], 'center': [31.0729871, -6.3429524], 'geometry': {'type': 'Point', 'coordinates': [31.0729871, -6.3429524]}, 'context': [{'id': 'country.8936', 'mapbox_id': 'dXJuOm1ieHBsYzpJdWc', 'wikidata': 'Q924', 'short_code': 'tz', 'text': 'Tanzania'}]}, {'id': 'country.8936', 'type': 'Feature', 'place_type': ['country'], 'relevance': 1, 'properties': {'mapbox_id': 'dXJuOm1ieHBsYzpJdWc', 'wikidata': 'Q924', 'short_code': 'tz'}, 'text': 'Tanzania', 'place_name': 'Tanzania', 'bbox': [29.390778, -11.761345, 40.5624687, -0.985788], 'center': [34.8234540265851, -6.27035338020445], 'geometry': {'type': 'Point', 'coordinates': [34.8234540265851, -6.27035338020

In [2]:
import pandas as pd
crop_price_trends = pd.read_csv("data/crop_prices_trend.csv")

In [3]:
crop_price_trends["commodity"].unique()

array(['Maize', 'Rice', 'Beans', 'Maize (white)', 'Maize flour (white)',
       'Potatoes (Irish)', 'Sweet potatoes', 'Wheat flour',
       'Fish (fresh)', 'Meat (beef, with bones)',
       'Meat (beef, without bones)', 'Meat (goat)', 'Sugar',
       'Oil (cotton)', 'Oil (sunflower)', 'Beans (dry)', 'Beans (green)',
       'Beans (spotted)', 'Soybeans', 'Cabbage', 'Carrots', 'Lettuce',
       'Onions', 'Tomatoes', 'Tomatoes (bitter)', 'Fuel (diesel)',
       'Fuel (petrol)', 'Millet (bulrush)', 'Millet (finger)', 'Sorghum',
       'Wheat'], dtype=object)

In [ ]:
'Maize', 'Rice', 'Beans''Wheat''Sorghum','Millet (bulrush)', 'Millet (finger)','Potatoes (Irish)'